In [159]:
import pandas as pd 
%matplotlib inline
import glob
import os.path

## import_coords_and_index
This function will take in the following parameters:

1) metadata_df= metadata from make_metadata()function above. Currently this must already be read into the notebook

2) path_to_coords= path to the folder containing the 96 to 384 coordinate conversation 

3) path_to_i7_384 = path to folder containing i7_indexes from 384 well plate 

4) path_to_i5_384 = path to folder containing the i5_indexes from 384 well plate 

5) i7_384_coord = coordinate from your i7 384 well plate you want to draw from 

6) i7_keyname = name of csv for the particular i7 index plate you are using

7) i5_384_coord = coordinate from your i5 384 well plate you want to draw from 

8) i5_keyname = name of csv for the particular i5 index plate you are using

9) well_type = default is "well_id_96"

10) path_to_data=path to folder where you will store your final table

11) date= date to be appended to your final sheet 

12) name= name of the final file, defaul is "metadata_indexes"

In [161]:
## make classes for accessing paths, metadat and indices

class MetaData:
    RETURNED_DATA="returned_data"
    
    def __init__(self, metadata_csv):
        self.path = "{a}/{b}".format(a=MetaData.RETURNED_DATA, b=metadata_csv)
        self.name = os.path.splitext(metadata_csv)[0]
    def make_df(self):
        return pd.read_csv(self.path).drop("Unnamed: 0", axis=1)
    def col_names(self):
        return pd.read_csv(self.path).columns  
    
class PathTo: 
    COORD_CONVS='coordinate_conversion_maps'
    i7_INDEXES='i7_indexes_plates'
    i5_INDEXES='i5_indexes_plates'
    RETURNED_DATA="returned_data"

class IndexPlateChoices: 
    def __init__(self, i7_plate_name, i7_chosen_coord, i5_plate_name, i5_chosen_coord):
        self.i7_plate_name = i7_plate_name
        self.i5_plate_name = i5_plate_name
        self.i7_chosen_coord = i7_chosen_coord
        self.i5_chosen_coord = i5_chosen_coord


In [162]:
## define paths and index selections and get metadata 
data_paths_20180607 = PathTo()
chosen_indices_20180607 = IndexPlateChoices("i7_index_384", "coord1_384", "i5_index_384", "coord1_384")
meta_data_20180607 = MetaData("20180529_metadata.csv")

In [158]:

def convert_csv_to_df(path_to_data):
    all_data=glob.glob("{}/*.csv".format(path_to_data))
    
    dict_of_all_data={}

    for file in all_data:
        file_name=os.path.splitext(os.path.basename(file))[0]
        df=pd.read_csv(file)
        dict_of_all_data[file_name]=df
    return dict_of_all_data

def import_coords_and_index(metadata_df,
                            data_paths,
                            chosen_indices,
                            well_type, 
                            date, 
                            saved_csv_name="metadata_w_indexes"):
    
    dict_i7_indices = convert_csv_to_df(data_paths.i7_INDEXES) ## could also so PathTo.i7_INDEXES bc all constants ( cool)
    dict_i5_indices = convert_csv_to_df(data_paths.i5_INDEXES)
    dict_of_coords = convert_csv_to_df(data_paths.COORD_CONVS)
    
    
    i7_selected_indices=dict_of_coords[chosen_indices.i7_chosen_coord].merge(dict_i7_indices[chosen_indices.i7_plate_name],left_on=chosen_indices.i7_chosen_coord, right_on=chosen_indices.i7_plate_name)
    i5_selected_indices=dict_of_coords[chosen_indices.i5_chosen_coord].merge(dict_i5_indices[chosen_indices.i5_plate_name],left_on=chosen_indices.i5_chosen_coord, right_on=chosen_indices.i5_plate_name)
    i7_metadata_merged=i7_selected_indices.merge(metadata_df, on=well_type)
    i5_i7_metadata_merged=i7_metadata_merged.merge(i5_selected_indices,on=well_type)
    
    i5_i7_metadata_merged.to_csv("{a}/{b}_{c}.csv".format(a=data_paths.RETURNED_DATA, b=date, c=saved_csv_name))
    return i5_i7_metadata_merged
    
experiment=import_coords_and_index(metadata_df = meta_data_20180607.make_df(),
                            data_paths = data_paths_20180607,
                            chosen_indices = chosen_indices_20180607,
                            well_type ="well_id_96", 
                            date = "20180607", 
                            saved_csv_name="metadata_with_indices")
experiment  

,well_id_96,coord1_384_x,i7_index_384,I7_Index_ID,i7_index_sequence,cell_number,experimental_or_control,nuclease,nucleofection_buffer,coord1_384_y,i5_index_384,i5_index_ID,i5_index_sequence
0,A1,A1,A1,i7-00001,CTGATCGT,1000000,exp,spCas9,P1,A1,A1,i5-00001,ACGATCAG
1,A2,A3,A3,i7-00003,TGAGCTAG,1000000,exp,spCas9,P1,A3,A3,i5-00003,CTAGCTCA
2,A3,A5,A5,i7-00005,CTTGTCGA,1000000,exp,spCas9,P1,A5,A5,i5-00005,TCGACAAG
3,A4,A7,A7,i7-00049,CGCATGAT,1000000,exp,spCas9,P1,A7,A7,i5-00049,ATCATGCG
4,A5,A9,A9,i7-00009,CGGCTAAT,1000000,exp,spCas9,P2,A9,A9,i5-00009,ATTAGCCG
5,A6,A11,A11,i7-00011,GCAAGATC,1000000,exp,spCas9,P2,A11,A11,i5-00011,GATCTTGC
6,A7,A13,A13,i7-00013,TACGCTAC,1000000,exp,spCas9,P2,A13,A13,i5-00013,GTAGCGTA
7,A8,A15,A15,i7-00015,AGAGTAGC,1000000,exp,spCas9,P2,A15,A15,i5-00015,GCTACTCT
8,A9,A17,A17,i7-00017,GACGATCT,1000000,exp,spCas9,P3,A17,A17,i5-00017,AGATCGTC
9,A10,A19,A19,i7-00019,CTTAGGAC,1000000,exp,spCas9,P3,A19,A19,i5-00019,GTCCTAAG


In [33]:
guides_ordered=pd.read_csv("guide_seq_tidy.csv")
guides_ordered
experiment_and_guides=experiment.merge(guides, on="well_id_96")
guides_ordered

,well_id_96,guide plate name,guide_description,guide_sequence
0,A1,oGEp12_01,vanOverbeek2016_01,GCTGATGTAGTCACTCTTGA
1,A2,oGEp12_02,vanOverbeek2016_02,AGTGTGCATTGCCACCTCAG
2,A3,oGEp12_03,vanOverbeek2016_03,AGATGCAGGCTGCAGATGCC
3,A4,oGEp12_04,vanOverbeek2016_04,GGCCCTCGGCGGCGGCTCCC
4,A5,oGEp12_05,vanOverbeek2016_05,ACAGAACATGTTGTTATAGA
5,A6,oGEp12_06,vanOverbeek2016_06,ACAGATTCTGGAAAGCTCCT
6,A7,oGEp12_07,vanOverbeek2016_07,GGTCAACAAGTGGAACTCTA
7,A8,oGEp12_08,vanOverbeek2016_08,GGAGTATTTCAGGGGCTCTT
8,A9,oGEp12_09,vanOverbeek2016_09,GGGAGGGCTGTGCTGCTAGT
9,A10,oGEp12_10,vanOverbeek2016_10,GCAACTTGAAATTATATCTG


In [34]:
VO_guides=pd.read_csv("VO_guides_2016.csv").dropna()
VO_guides

,Spacer,Spacer sequence,Genomic location of spacer (hg19)
0,1.0,CCCTCAAGAGTGACTACATCAGC,chr11:836293-836316
1,2.0,AGTGTGCATTGCCACCTCAGTGG,chr4:145041692-145041714
2,3.0,CCGGGCATCTGCAGCCTGCATCT,chr15:90348654-90348677
3,4.0,CCTGGGAGCCGCCGCCGAGGGCC,chr17:40440261-40440283
4,5.0,CCATCTATAACAACATGTTCTGT,chrX:138644010-138644032
5,6.0,CCGAGGAGCTTTCCAGAATCTGT,chrX:66765045-66765067
6,7.0,CCTTAGAGTTCCACTTGTTGACC,chrX:138643957-138643979
7,8.0,CCCAAGAGCCCCTGAAATACTCC,chr8:39871134-39871156
8,9.0,CCCACTAGCAGCACAGCCCTCCC,chr1:183218409-183218431
9,10.0,CCACAGATATAATTTCAAGTTGC,chr7:50526196-50526218


### Convert your work to a python script. 

`! jupyter nbconvert --to script [yournotebookname].ipynb`

In [55]:
! jupyter nbconvert --to script 20180605_Giana_Aaron_labmate_96totidy_addindex.ipynb

[NbConvertApp] Converting notebook 20180605_Giana_Aaron_labmate_96totidy_addindex.ipynb to script
[NbConvertApp] Writing 4504 bytes to 20180605_Giana_Aaron_labmate_96totidy_addindex.py


In [35]:
vo_guides = VO_guides["Spacer sequence"]

In [45]:
COMPLEMENTS = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}

def rev_complement_seq(seq):
    return ''.join([COMPLEMENTS[bp]for bp in reversed(seq)])

In [39]:
ordered_guides = guides_ordered["guide_sequence"]

In [48]:
assert len(vo_guides) == len(ordered_guides)

for idx, vo_guide in enumerate(vo_guides):
    if vo_guide[-2:] == 'GG':
        assert vo_guide[:20] == ordered_guides[idx]
    elif vo_guide[:2] == 'CC':
        assert rev_complement_seq(vo_guide)[:20] == ordered_guides[idx]
    else:
        raise Exception("This should not happen.")

In [54]:
i = [int(g.split("_")[1]) for g in guides_ordered["guide plate name "]]
j = [int(e) for e in VO_guides["Spacer"]]
i == j

True